阿里云和国家气象信息中心联合推出

全国每平方公里每小时的气象实况和预报数据[产品](http://nature.data.aliyun.com/getoken.html)

当调用历史数据时，只能调用**最近三天**的。当调用预测数据时，可以调用**未来10天**的。

## 根据准确经纬度查询点数据API

In [21]:
import urllib2
import urllib
values = {
    'token': '1jkMvPoifu',
    'action': 'data_value_by_date_name_geometry',
    'name' : 'CLDAS-TEMP',
    'date': '20170712_080000',
    'geometry': '105.026,37.526'
}
args = urllib.urlencode(values)
response = urllib2.urlopen('http://nature.data.aliyun.com/api?' + args)
data = response.read()
print data

{
  "data": {
    "data_value_by_date_name_geometry": {
      "unit": "摄氏度(°C)",
      "value": "27.25231323242201",
      "date": "2017071208_GMT+8",
      "time_zone": "GMT+8"
    }
  }
}


## 查询矩形区域数据API

In [22]:
import urllib2
import urllib
values = {
 'token': '1jkMvPoifu',
 'action': ' data_export_by_date_name_geometry ',
 'name' : 'CMPA-PREP',
 'date': '20170712_080000',
 'geometry': '80.026,35.026,105.026,20.426',
 'format': 'tiff'
}
args = urllib.urlencode(values)
response = urllib2.urlopen('http://nature.data.aliyun.com/api?' + args)
data = response.read()
print data 

{
  "errors": [
    {
      "message": "Cannot query field \"undefined\" on type \"DataFileBlockExport\"."
    }
  ]
}


上述方法只能做天气预测，而不能查询特定时间的天气数据

思路：

1. 根据经纬度，定位到城市、县区，最终在[天气网](http://lishi.tianqi.com/)上查询该地点天气信息。
2. [百度地图开放平台—车联网API](http://lbsyun.baidu.com/index.php?title=car)

## 根据经纬度获取地址信息

In [97]:
import urllib
import hashlib
import json

lng_lat = '39.983424,116.322987'
def geocoding(lng_lat): 
    add = {}
    ak = 'WtPdP20GRKi9Ord9o8bGmfMYRYK3pgMZ'
    sk = 'G0GUQW7lx7bQBzw2wcGnPTVq8kCQkbNu'
    queryStr = '/geocoder/v2/?location=' + lng_lat + '&output=json&ak=' + ak

    # 对queryStr进行转码，safe内的保留字符不转换
    encodedStr = urllib.quote(queryStr, safe="/:=&?#+!$,;'@()*[]")
    # 在最后直接追加上yoursk
    rawStr = encodedStr + sk
    sn = hashlib.md5(urllib.quote_plus(rawStr)).hexdigest()
    url = 'http://api.map.baidu.com' + queryStr + '&sn=' + sn  # 最终提交的url
    info_json = json.loads(urllib.urlopen(url).read())  # 解析返回的json格式数据

    add['province'] = info_json['result']['addressComponent']['province']  # 省
    add['city'] = info_json['result']['addressComponent']['city']  # 市
    add['district'] = info_json['result']['addressComponent']['district']  # 区县
    add['street'] = info_json['result']['addressComponent']['street']  # 街
    
    return add

## 根据城市区县拼音获取天气

In [98]:
import requests
from lxml import etree

ymd = '20160811'
address = 'shantou'
def get_weather(address, ymd):
    wea = {}
    url  = 'http://' + address + '.tianqi.com/' + ymd + '.html'
    html = requests.get(url).text
    wea['day'] = etree.HTML(html).xpath('//*[@id="today"]/ul/li[2]')[1].text  # 白天天气
    wea['night'] = etree.HTML(html).xpath('//*[@id="today"]/ul/li[2]')[2].text  # 夜间天气
    wea['temp_high'] = etree.HTML(html).xpath('//*[@id="t_temp"]/font[1]')[0].text  # 最高气温
    wea['temp_low'] = etree.HTML(html).xpath('//*[@id="t_temp"]/font[2]')[0].text  # 最低气温
    wea['wind'] = etree.HTML(html).xpath('//*[@id="today"]/ul/li[5]')[0].text  # 风向风速
    return wea

## 根据区县中文对应网站拼音格式

In [134]:
import os
def get_city_dict():
    if os.path.exists('city_dict.txt'):
        with open('city_dict.txt', 'r') as fp:
            city_dict = json.load(fp)
    else:
        city_dict = {}
        url  = 'http://lishi.tianqi.com'
        html = requests.get(url).text
        chars = etree.HTML(html).xpath('//*[@id="tool_site"]/div[2]/ul')  # 每个字母
        for char in chars:
            citys = char.xpath('li/a/text()')  # 提取区县名
            pinyins = char.xpath('li/a/@href')  # 提取区县链接
            num = len(citys)
            assert len(pinyins) == num   
            for i in range(1,num):
                city_dict[citys[i]] = pinyins[i].split('/')[-2]  # 从区县链接中提取拼音
        with open('city_dict.txt', 'w') as fp:
            fp.write(json.dumps(city_dict))
    return city_dict

In [216]:
city_dict = get_city_dict()
citys = city_dict.keys()
def get_wea_fin(lng=23.2859, lat=116.7266, date='20170701'):
    lng_lat = str(lng) + ',' + str(lat)
    address = geocoding(lng_lat)
    province = address['province']  # 通过百度API提取的省名称
    city = address['city']  # 市
    district = address['district']  # 区县
    weather = {}
    if district in citys:
        weather = get_weather(city_dict[district], str(date))
    elif district[:-1] in citys:  # 天气网的部分区县没有以区县结尾，如，闵行区表示为闵行
         weather = get_weather(city_dict[district[:-1]], str(date))
    elif district == '' and city != '':  # 不存在区县信息，取城市信息
        weather = get_weather(city_dict[city], str(date))
    else:
        print '对不起，无法查到您给的经纬度信息，请输入中国大陆范围内经纬度信息。'
    return str(date), address, weather

In [223]:
weather = get_wea_fin(31.1219830, 121.3743330, '20141207')
print weather[0], weather[1]['province'], weather[1]['city'], weather[1]['district'], weather[1]['street']
for k,v in weather[2].items():
    print k,v

20141207 上海市 上海市 闵行区 莘凌路
temp_low 10℃
temp_high 3℃
day 晴
wind 西北风转北风3-4级
night 多云
